# TESS Archive Tutorial
### By Susan E. Mullally

## Querying the TIC

In [3]:
from astroquery.mast import Catalogs
import numpy as np

## Astroquery Catalog Cone Search on TIC

We will now use the Catalogs `query_object` function to do a cone search centered on BPM 31594 and return the rows from the TIC catalog.  The function makes use of MAST's name resolver to convert a target name (as a string) into coordinates (Right Ascension and Declination).  Many types of ID's are possible, including any string that can be resolved by Simbad or NED, as well as any target in the Kepler Input Catalog, K2 EPIC Catalog, or the TESS Input Catalog.  The function also can accept a search radius size (specified in degrees), and it needs to know which catalog you want to query.  Catalog options are listed on [this page](https://astroquery.readthedocs.io/en/latest/mast/mast.html#catalog-queries).


In [4]:
target_name = "BPM 31594"  #Pulsating White Dwarf Star
search_radius_deg = 0.1

# Query the TESS Input Catalog centered on HD 209458 with a 0.2 degree radius.
catalogTIC = Catalogs.query_object(target_name, radius=search_radius_deg, catalog="TIC")

# Print out the number of returned rows.
print("Number of TIC objects within %f deg of %s: %u" % (search_radius_deg, target_name, len(catalogTIC)))

Number of TIC objects within 0.100000 deg of BPM 31594: 84


In [6]:
#Print out the columns
catalogTIC.columns

<TableColumns names=('ID','ra','dec','pmRA','pmDEC','Tmag','objType','typeSrc','version','HIP','TYC','UCAC','TWOMASS','SDSS','ALLWISE','GAIA','APASS','KIC','POSflag','e_pmRA','e_pmDEC','PMflag','plx','e_plx','PARflag','gallong','gallat','eclong','eclat','Bmag','e_Bmag','Vmag','e_Vmag','umag','e_umag','gmag','e_gmag','rmag','e_rmag','imag','e_imag','zmag','e_zmag','Jmag','e_Jmag','Hmag','e_Hmag','Kmag','e_Kmag','TWOMflag','prox','w1mag','e_w1mag','w2mag','e_w2mag','w3mag','e_w3mag','w4mag','e_w4mag','GAIAmag','e_GAIAmag','e_Tmag','TESSflag','SPFlag','Teff','e_Teff','logg','e_logg','MH','e_MH','rad','e_rad','mass','e_mass','rho','e_rho','lumclass','lum','e_lum','d','e_d','ebv','e_ebv','numcont','contratio','disposition','duplicate_id','priority','dstArcSec')>

In [8]:
where_closest = np.argmin(catalogTIC['dstArcSec'])

print("Closest TIC ID to %s: TIC %s, separation of %f deg. and a TESS mag. of %f"%
      (target_name, catalogTIC['ID'][where_closest], catalogTIC['dstArcSec'][where_closest],
      catalogTIC['Tmag'][where_closest]))

ticid = catalogTIC['ID'][where_closest]

Closest TIC ID to BPM 31594: TIC 101014997, separation of 0.009714 deg. and a TESS mag. of 15.079000


### Problem 1.  TIC Query
Determine the ticid of L19-2, a pulsating white dwarf with V magnitude of 13.7.

In [15]:
target_name = "L19-2"
#Your code here


### Extra Credit. Direct TIC Query
Directly filter the TIC by the integer tic ID to look up the Tess Mag and RA and Dec of that object in the catalog. This is done by using the Catalogs query_criteria and setting the catalog to "TIC".  Take a look at the examples on the [astroquery documentation page](https://astroquery.readthedocs.io/en/latest/mast/mast.html) to see if you can figure out how to filter the TIC by "ID".

In [14]:
#Your code here


## Finding Lightcurves at MAST

In [16]:
from astroquery.mast import Observations
from astropy.io import fits

## Query the MAST CAOM Database

We want to retrieve TESS timeseries data (lightcurve and target pixel files) for the Guest Investigator program G011113 from Sector 1 (PI: JJ Hermes). We will need to query the MAST holdings database for the observations.  First we will simply count the number of observations and then we will request a table of those observations. 

All the filter names are listed on the [MAST CAOM Field Description Page](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html). Or, can be found by hovering over the column names after doing a search in the MAST portal.

The filters we will need to use are, 
- **obs_collection** to specify that we want TESS data
- **dataproduct_type** to specify that we want timeseries data
- **sequence_number** to specify that we want sector 1  
- **propsal_id** to specify the GI program number.  

Hint! More than one GI can propose for the same target so we need wild cards around the name of the program in our query.

In [26]:
#Count the number of observations with query_criteria_count
obsCount = Observations.query_criteria_count(obs_collection = "TESS",
                                             dataproduct_type = ["timeseries"],
                                             sequence_number = 1,
                                             proposal_id = "*G011113*")
print("Number of Observations: %i" % obsCount)

#Print the first 5 with query_criteria
obsTable = Observations.query_criteria(obs_collection = "TESS",
                                       dataproduct_type = ["timeseries"],
                                       sequence_number = 1,
                                       proposal_id = "*G011113*")
obsTable[0:5]['obsid','proposal_id','obs_id']

Number of Observations: 109


obsid,proposal_id,obs_id
str11,str23,str47
17000013114,G011113,tess2018206045859-s0001-0000000161154061-0120-s
17000009148,G011113,tess2018206045859-s0001-0000000140045537-0120-s
17000002478,G011113,tess2018206045859-s0001-0000000031674330-0120-s
17000000402,G011113,tess2018206045859-s0001-0000000079353860-0120-s
17000014360,G011113,tess2018206045859-s0001-0000000325341811-0120-s


## Retrieve the list of Data Products

Next we use astroquery to retrieve the list of data products that are associated with each observation.  

We will only ask for the data product list associated with the first two in obsTable.  The `[0:2]` can be removed from the code below to get all the observations.

In [33]:
dataProducts = Observations.get_product_list(obsTable[0:2])
dataProducts.colnames

['obsID',
 'obs_collection',
 'dataproduct_type',
 'obs_id',
 'description',
 'type',
 'dataURI',
 'productType',
 'productGroupDescription',
 'productSubGroupDescription',
 'productDocumentationURL',
 'project',
 'prvversion',
 'proposal_id',
 'productFilename',
 'size',
 'parent_obsid']

In [28]:
#Print off the list of observations
dataProducts['obsID', 'productFilename', 'description']

obsID,productFilename,description
str11,str63,str27
17000009148,tess2018206190142-s0001-s0001-0000000140045537-00106_dvr.pdf,full data validation report
17000009148,tess2018206190142-s0001-s0001-0000000140045537-00106_dvr.xml,full data validation report
17000009148,tess2018206190142-s0001-s0001-0000000140045537-01-00106_dvs.pdf,TCE summary report
17000009148,tess2018206190142-s0001-s0001-0000000140045537-00106_dvt.fits,Data validation time series
17000009148,tess2018206045859-s0001-0000000140045537-0120-s_lc.fits,Light curves
17000009148,tess2018206045859-s0001-0000000140045537-0120-s_tp.fits,Target pixel files
17000013114,tess2018206045859-s0001-0000000161154061-0120-s_lc.fits,Light curves
17000013114,tess2018206045859-s0001-0000000161154061-0120-s_tp.fits,Target pixel files


## Download Light curves
<a id="download"></a>
We limit our list of data products to just those with the **description** set to "Light curves" because we just want this type of data file. We then download those products. The manifest is an array indicating the download location of the files. 

In [29]:
want = dataProducts['description'] == "Light curves"
manifest = Observations.download_products(dataProducts[want])
print(manifest)

                                                         Local Path                                                         ...
--------------------------------------------------------------------------------------------------------------------------- ...
./mastDownload/TESS/tess2018206045859-s0001-0000000140045537-0120-s/tess2018206045859-s0001-0000000140045537-0120-s_lc.fits ...
./mastDownload/TESS/tess2018206045859-s0001-0000000161154061-0120-s/tess2018206045859-s0001-0000000161154061-0120-s_lc.fits ...


### Problem 2.
Looks like the TESS SPOC pipeline found a TCE (Threshold Crossing Event) when it searched these data for indications of periodic tansits. When a TCE is found, the SPOC pipeline also creates a DV time series FITS file.  Download just the DV time series file for the first observation.

Some of the other fields you might consider filtering on are listed [here](https://outerspace.stsci.edu/display/TESS/7.0+-+Tips+and+Tricks+to+Getting+TESS+Data+At+MAST).

### Bonus Problem.
How many of the sector 1 targets produced a TCE.

In [37]:
#Your code here.

#Bonus Problem


## Exploring and Plotting the Light Curve File


In [ ]:
#Answers to Astroquery Search.
#Problem 1.
Catalogs.query_object(target_name, radius=search_radius_deg, catalog="TIC")["ID"][0]
#It always returns sorted by distance.  Be careful with high proper motion objects.

#Bonus Problem 1.
Catalogs.query_criteria(catalog="Tic",ID=101014997)['Tmag']


In [ ]:
#Answers to Data Product Query
manifest = Observations.download_products(dataProducts[dataProducts['productSubGroupDescription'] == 'DVT'])

#Bonus Problem
allDataProd = Observations.get_product_list(obsTable)
want = allDataProd['description'] == "Data validation time series"
print("Number of DVT files:  %u" % len(want[want]))